In [0]:
%pip install statsmodels
%matplotlib inline

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
%run /Users/ebizindavyi@delhaize.be/Data_preparation/Parse_xls_files

In [0]:
date= '2022-07-01'
training_period=90
horizon_begin=15
days_to_forecast=7
best_model=(1, 0, 0)
seasonal_order=(2, 0, 1, 7)

## Load Libraries

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.tsa.arima.model import ARIMA
import datetime as dt
from datetime import datetime, timedelta

## Load the inbound data

In [0]:
inbound_2022_dir = '/dbfs/mnt/dataplatform/acc/DataScience/sandbox/ebiz/Inbound_FC/data/input/Ecom-Ops-status_2022.xlsx'

In [0]:
inbound_2022_df = create_master_df(inbound_2022_dir)

### Create yearly time series
needed to fill in date gaps

In [0]:
calendar_2022 = pd.DataFrame({'Dates':pd.date_range(dt.datetime(2022,1,3), periods=363, freq='d')})
inbound_2022_dry = calendar_2022.merge(inbound_2022_df, on='Dates',how='left').fillna(0).set_index('Dates').copy()
inbound_2022_dry = inbound_2022_dry.groupby(['Dates'])['Dry Actuals'].sum()
inbound_2022_dry

Out[18]: Dates
2022-01-03    84224.0
2022-01-04    49564.0
2022-01-05    45890.0
2022-01-06    26976.0
2022-01-07    37044.0
               ...   
2022-12-27    63764.0
2022-12-28    47680.0
2022-12-29    39552.0
2022-12-30    61119.0
2022-12-31    46818.0
Name: Dry Actuals, Length: 363, dtype: float64

## ARIMA model with fitted parameters

In [0]:
start_train_date = datetime.strptime(date,'%Y-%m-%d').date()
end_train_date = start_train_date + timedelta(days=training_period-1)
endog = inbound_2022_dry[start_train_date:end_train_date]

In [0]:
%%time 
arima_mod = ARIMA(endog, order=best_model, seasonal_order=seasonal_order, trend=None, freq=None)

CPU times: user 1.3 ms, sys: 3.97 ms, total: 5.27 ms
Wall time: 5.98 ms


/databricks/python/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/databricks/python/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/databricks/python/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)


In [0]:
arima_mod_fit=arima_mod.fit()

###check the parameters

###Prediction
First period starts from 14th day from the end_train_date until the 15th day 


In [0]:
#predict, hints: 1)specify the train period with start and end date

horizon_start= end_train_date+timedelta(days=horizon_begin)
horizon_end=horizon_start+timedelta(days=days_to_forecast-1)
arima_pred=arima_mod_fit.predict(start=horizon_start, end=horizon_end)


In [0]:
arima_pred

Out[23]: 2022-10-13    45951.427704
2022-10-14    50975.516046
2022-10-15    51324.100041
2022-10-16    14475.565246
2022-10-17    88409.474137
2022-10-18    61466.483229
2022-10-19    60445.087121
Freq: D, Name: predicted_mean, dtype: float64

In [0]:
initial_date = '2022-04-03'
end_date = '2022-12-31'
fc_freq = 7

In [0]:
# forecasting date sequences is the the list of the start date of each of our forecasting period 
forecast_date_seq = pd.date_range(start=initial_date, end=end_date, periods=fc_freq)

In [0]:
initial_date= datetime.strptime(initial_date, '%Y-%m-%d')

forecast_date_seq= pd.date_range(start=initial_date, end=end_date, periods=fc_freq)
backtest_results=pd.DataFrame()

for start_test_date in forecast_date_seq:
  arima_mod = ARIMA(endog, order=best_model, seasonal_order=seasonal_order, trend=None, freq=None)
  end_test_date =initial_date + fc_freq-1

  start_train_date= start_test_date-training_days-1
  end_train_date=start_train_date+training_days

  endog=inbound_2022_dry[start_train_date:end_train_date]
  ARIMA
  # backtest_results = backtest_results.append(forecast_df)
  pass

end_test_date =start_test_date + fc_horizon_end

start_train_date= start_test_date-training_days-1
end_train_date=start_train_date+training_days

endog=inbound_2022_dry[start_train_date:end_train_date]

/databricks/python/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/databricks/python/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/databricks/python/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)


---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
File <command-846493552946166>:8
      6 for start_test_date in forecast_date_seq:
      7   arima_mod = ARIMA(endog, order=best_model, seasonal_order=seasonal_order, trend=None, freq=None)
----> 8   end_test_date =initial_date + fc_freq-1
     10   start_train_date= start_test_date-training_days-1
     11   end_train_date=start_train_date+training_days

TypeError: unsupported operand type(s) for +: 'datetime.datetime' and 'int'